## Teleportation experiment from paper by Fedortchenko

This notebook uses Qubiter to illustrate a pedagogical Teleportation 
experiment performed by S. Fedortchencko on IBM Quantum Experience, 
and described by him in the paper

>https://arxiv.org/abs/1607.02398

$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$

First change your working directory to the qubiter directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


In [2]:
from SEO_writer import *
from SEO_simulator import *
from StateVec import *
import numpy as np

Number of qubits is 3.
Note that we use "bit" for both qbits and cbits

In [3]:
num_bits = 3

Use a trivial circuit embedder that embeds 3 qubits into same 3 qubits

In [4]:
emb = CktEmbedder(num_bits, num_bits)

Open a writer, tell it where to write to.
We will use zero bit last (ZL) convention

In [5]:
file_prefix = 'io_folder/teleportation-fedor'
wr = SEO_writer(file_prefix, emb)

Write partial circuit that creates
a state at qbit 0 that will be teleported to qbit 2.

In [6]:
z_axis = 3
def init_ckt():
    wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)
    wr.write_one_bit_gate(0, OneBitGates.rot_ax, [-np.pi/8, z_axis])  # T(0)
    wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)
    wr.write_one_bit_gate(0, OneBitGates.rot_ax, [-np.pi/4, z_axis])  # S(0)
init_ckt()
wr.close_files()

init_st_vec = StateVec.get_standard_basis_st_vec([0, 0, 0])
sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
StateVec.describe_st_vec_dict(sim.cur_st_vec_dict, print_st_vec=True)

*********branch= pure
state vector:
[[[ 0.65328148-0.65328148j  0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]

 [[ 0.27059805-0.27059805j  0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]]
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.8535533905932734, 0.14644660940672616),
 1: (0.99999999999999956, 0.0),
 2: (0.99999999999999956, 0.0)}


Now run the whole circuit.
If you want to see the state vector after only first n gates,
where n is less than total number of gates, just
comment out all gates from n+1 to the final one. You can comment
multiple lines with control-slash applied to a selected block. 
control-slash also uncomments a selected, commented block.

In [7]:
wr = SEO_writer(file_prefix, emb)
init_ckt()
wr.write_one_bit_gate(2, OneBitGates.had2)  # H(2)

control_pos = 2
target_pos = 1
trols = Controls.new_knob(num_bits, control_pos, kind=True)
wr.write_controlled_one_bit_gate(target_pos, trols, OneBitGates.sigx)

control_pos = 0
target_pos = 1
trols = Controls.new_knob(num_bits, control_pos, kind=True)
wr.write_controlled_one_bit_gate(target_pos, trols, OneBitGates.sigx)

wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)

wr.write_MEAS(tar_bit_pos=0, kind=0)
wr.write_MEAS(tar_bit_pos=1, kind=0)
wr.close_files()

init_st_vec = StateVec.get_standard_basis_st_vec([0, 0, 0])
sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
StateVec.describe_st_vec_dict(sim.cur_st_vec_dict, print_st_vec=True)


*********branch= pure
state vector:
[[[ 0.32664074-0.32664074j  0.13529903-0.13529903j]
  [ 0.00000000+0.j          0.00000000+0.j        ]]

 [[ 0.00000000+0.j          0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]]
total probability of state vector (=one if no measurements)= 0.25
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.24999999999999978, 0.0),
 1: (0.24999999999999978, 0.0),
 2: (0.21338834764831827, 0.036611652351681512)}


Look in files
* <a href="../io_folder/teleportation-fedor_3_eng.txt">../io_folder/teleportation-fedor_3_eng.txt</a>
* <a href="../io_folder/teleportation-fedor_3_ZLpic.txt">../io_folder/teleportation-fedor_3_ZLpic.txt</a>

to see the quantum circuit that was generated

If you do the algebra, the final state vector should be

$
\begin{array}{c}\ket{0}\\ \ket{0} \\ (\frac{cc}{2} \ket{0} + \frac{ss}{2}\ket{1}\end{array}+
\begin{array}{c}\ket{0} \\ \ket{1} \\ (\frac{cc}{2} \ket{0} + \frac{ss}{2}\ket{1})\end{array}+
\begin{array}{c}\ket{1} \\ \ket{0} \\ (\frac{cc}{2} \ket{0} - \frac{ss}{2}\ket{1})\end{array}+
\begin{array}{c}\ket{1} \\ \ket{1} \\ (\frac{-ss}{2} \ket{0} + \frac{cc}{2}\ket{1})\end{array}
$

where cc = cos(\pi/8) and ss = sin(\pi/8), and where the 
first, second,
third
row corresponds to qubit 0, 1, 2